In [1]:
import pandas as pd
import itertools

import torch
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t6_8M_UR50D")
device = torch.device("cuda")

Downloading: "https://github.com/facebookresearch/esm/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t6_8M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D-contact-regression.pt


In [32]:
CDR_seq = pd.read_csv('CDR_seq_df.csv', index_col=0)
CDR_seq_melt = CDR_seq.melt()
CDR_seq_melt['name'] = ['_'.join(i) for i in itertools.product(CDR_seq.columns, CDR_seq.index)]

In [33]:
CDR_seq_melt

,variable,value,name
0,L1,TGTSSDVGGYNYVS,L1_7sue
1,L1,RASQSITGNWVA,L1_5js9
2,L1,ASGFAFSSFGM,L1_6lfm
3,L1,ASGFAFSSFGM,L1_6lfo
4,L1,RASQEVGESELA,L1_7su1
...,...,...,...
14539,H3,SNWLDPIRGGS,H3_5ock
14540,H3,NNEGGV,H3_4d9q
14541,H3,ECEEWWSDDFGKQLPCAKSRGGLVGIADNDLRED,H3_6vtt
14542,H3,FAAYNSLDVGMEGL,H3_6vtu


In [34]:
CDR_seq_melt[CDR_seq_melt['value'].isna()]
# 11 структур, в которых нет каких-то из CDR

,variable,value,name
3061,L2,NaN,L2_7ey0
3731,L2,NaN,L2_5d70
3732,L2,NaN,L2_5d71
4727,L2,NaN,L2_4hkx
8786,H1,NaN,H1_6idk
8788,H1,NaN,H1_6idl
8906,H1,NaN,H1_6idi
9162,H1,NaN,H1_4lsq
9163,H1,NaN,H1_4lsp
9165,H1,NaN,H1_4lsr


In [35]:
CDR_seq_melt.dropna(inplace=True)

In [28]:
CDR_seq_melt

,variable,value,name
0,L1,TGTSSDVGGYNYVS,L1_7sue
1,L1,RASQSITGNWVA,L1_5js9
2,L1,ASGFAFSSFGM,L1_6lfm
3,L1,ASGFAFSSFGM,L1_6lfo
4,L1,RASQEVGESELA,L1_7su1
...,...,...,...
14539,H3,SNWLDPIRGGS,H3_5ock
14540,H3,NNEGGV,H3_4d9q
14541,H3,ECEEWWSDDFGKQLPCAKSRGGLVGIADNDLRED,H3_6vtt
14542,H3,FAAYNSLDVGMEGL,H3_6vtu


In [42]:
data = list(zip(CDR_seq_melt['name'], CDR_seq_melt['value']))

batch_converter = alphabet.get_batch_converter()
batch_labels, batch_strs, batch_tokens = batch_converter(data)

In [46]:
model = model.to(device)
batch_tokens = batch_tokens.to(device)

In [48]:
seq_mean_embeddings = torch.zeros((len(batch_strs), 320), device=device)

In [72]:
# colab что-то ругается на память, поэтому батчами
batch_size = 500
dataloader = torch.utils.data.DataLoader(
    batch_tokens,
    batch_size=batch_size,
)

In [75]:
for i, batch in enumerate(dataloader):
    with torch.no_grad():
        results = model(batch, repr_layers=[6])
    seq_mean_embeddings[i * batch_size : (i+1) * batch_size] = results["representations"][6].mean(1)

In [77]:
seq_mean_embeddings = seq_mean_embeddings.cpu().numpy()

In [80]:
esm_df = pd.DataFrame(seq_mean_embeddings, index=batch_labels)

In [ ]:
esm_df.to_csv('СDR_seq_esm_df.csv')